In [3]:
import os
import json
import shutil
from PIL import Image
from collections import defaultdict

# 경로 설정
pred_json = 'C:/Users/user/DisasterDetection_곽용진/Code/runs/detect/disaster_detection_test7/predictions.json'
label_dir = 'C:/Users/user/Downloads/dataset/dataset/labels/test'
image_dir = 'C:/Users/user/Downloads/dataset/dataset/images/test'
fp_dir = 'C:/Users/user/hyun_codes/FP'
fn_dir = 'C:/Users/user/hyun_codes/FN'
os.makedirs(fp_dir, exist_ok=True)
os.makedirs(fn_dir, exist_ok=True)

def iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    xi1 = max(x1, x2)
    yi1 = max(y1, y2)
    xi2 = min(x1 + w1, x2 + w2)
    yi2 = min(y1 + h1, y2 + h2)
    inter_w = max(0, xi2 - xi1)
    inter_h = max(0, yi2 - yi1)
    inter_area = inter_w * inter_h
    box1_area = w1 * h1
    box2_area = w2 * h2
    union_area = box1_area + box2_area - inter_area
    return inter_area / union_area if union_area > 0 else 0

def load_gt_boxes(label_path, img_w, img_h):
    boxes = []
    if not os.path.exists(label_path):
        return boxes
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) < 5:
                continue
            cls, x_c, y_c, w, h = map(float, parts)
            x = (x_c - w / 2) * img_w
            y = (y_c - h / 2) * img_h
            boxes.append({'cls': int(cls), 'bbox': [x, y, w * img_w, h * img_h]})
    return boxes

# predictions.json 파싱
with open(pred_json, 'r') as f:
    preds = json.load(f)

pred_dict = defaultdict(list)
for pred in preds:
    img_id = pred['image_id']
    pred_dict[img_id].append(pred)

for img_name in os.listdir(image_dir):
    if not img_name.lower().endswith(('.jpg', '.png', '.jpeg')):
        continue
    base = os.path.splitext(img_name)[0]
    label_path = os.path.join(label_dir, base + '.txt')
    img_path = os.path.join(image_dir, img_name)
    img = Image.open(img_path)
    img_w, img_h = img.size

    gt_boxes = load_gt_boxes(label_path, img_w, img_h)
    pred_boxes = pred_dict.get(base, [])

    used_gt = set()
    used_pred = set()
    # 1:1 그리디 매칭 (클래스도 비교)
    for i, gt in enumerate(gt_boxes):
        best_j = -1
        best_iou = 0
        for j, pred in enumerate(pred_boxes):
            if j in used_pred:
                continue
            if int(pred['category_id']) != gt['cls']:
                continue  # 클래스 불일치시 매칭하지 않음
            current_iou = iou(gt['bbox'], pred['bbox'])
            if current_iou > best_iou:
                best_iou = current_iou
                best_j = j
        if best_iou >= 0.5:
            used_gt.add(i)
            used_pred.add(best_j)
    # FN: 매칭 안 된 GT
    if any(i not in used_gt for i in range(len(gt_boxes))):
        shutil.copy2(img_path, fn_dir)
        print(f'FN: {img_name}')
    # FP: 매칭 안 된 예측
    if any(j not in used_pred for j in range(len(pred_boxes))):
        shutil.copy2(img_path, fp_dir)
        print(f'FP: {img_name}')


FP: -_jpg.rf.db73f303f8bd46ae32fc5ae6c748a5ed.jpg
FP: 000-1023-_jpg.rf.a4e323f6f3bd78bdb054b2c493cd9172.jpg
FP: 000-1037-_jpg.rf.d5ceff33564d74c7ef0f98269461ce48.jpg
FP: 00038_jpg.rf.035cc9d417791d9453f0e4e07b11171a.jpg
FP: 00043_jpg.rf.ef44d31cf90ec1e899191c4607238654.jpg
FN: 1100604-12_jpg.rf.d637614db8c19111f91b13fa25c70226.jpg
FP: 1100604-12_jpg.rf.d637614db8c19111f91b13fa25c70226.jpg
FP: 1100604-433_jpg.rf.de1e30e582f5d7ff25360fc6bc810dee.jpg
FP: 1100604-452_jpg.rf.6aa2eb3dbfccbe8888d16f476eeb5142.jpg
FP: 1100604-600_jpg.rf.0ec0a8156436bfce4cf2a4db3feb990c.jpg
FP: 113_png_jpg.rf.1f57b8ee7db3d71fdf9b98af930a9cf6.jpg
FP: 1170_McCook-Reservoir-time-lapse-February-Fill-5kJ2Nbr_shc-_jpg.rf.7ea6a2946b1fd8d572c3a4b11d3c64b7.jpg
FN: 1231914853_x_jpg.rf.4cdaeb88559a5bbad45b2031ca71ab5b.jpg
FP: 1231914853_x_jpg.rf.4cdaeb88559a5bbad45b2031ca71ab5b.jpg
FN: 12870_Hurricane-Harvey-Flood-Houston-TX-Meyerland-Neighborhood-August-27-2017-Garage-Time-Lapse-ZOpWO7rJbtU-_jpg.rf.77eedaee6dacac56b3d048

FP: CarFiresVed_mp4-1084_jpg.rf.e44829d7ed25d12679d15158b5a759e8.jpg
FP: CarFiresVed_mp4-1091_jpg.rf.8a5fea160509ae236e4db266bcc6b6e3.jpg
FP: CarFiresVed_mp4-1095_jpg.rf.946b6bac2470764d0dc362b70583afe2.jpg
FP: CarFiresVed_mp4-1096_jpg.rf.13d620c284bddc7c417c2d198c8e8c8e.jpg
FP: CarFiresVed_mp4-1111_jpg.rf.5abe278034ec65bbb10a4819c281796c.jpg
FP: CarFiresVed_mp4-1113_jpg.rf.23d58de9b8ddabf055f47879c0de7c09.jpg
FP: CarFiresVed_mp4-1116_jpg.rf.7299ffbdf538662b208e5b76ce892c2d.jpg
FP: CarFiresVed_mp4-1117_jpg.rf.a61dc66ef10eabb530b00067c394edf8.jpg
FP: CarFiresVed_mp4-1122_jpg.rf.9a22850dcc58d7fa97f0ef73e5cf244e.jpg
FP: CarFiresVed_mp4-1133_jpg.rf.2bfd2ad71d563a80f28b795470aac190.jpg
FP: CarFiresVed_mp4-1134_jpg.rf.c4376b6addecca3f2b8ba94a2d026dc2.jpg
FP: ciliwung_2023Y_11M_18D_13H_jpg.rf.10bae6c0725f5cd3935d6c95c9000f26.jpg
FP: frame236_jpg.rf.fcf0b0c4ca6316ede1c5fd64dc24dd14.jpg
FP: frame266_jpg.rf.bf5866c9c23b9e50f27f9000061c88af.jpg
FP: frame85_jpg.rf.49ebdc22afed079eeac810bc93b36f97.

FP: Image_94_jpg.rf.b6031079120df529c49f27549ba76041.jpg
FP: Image_96_jpg.rf.6096fd9891ed107402cc25a94650f5f6.jpg
FP: Image_9_jpg.rf.2347fc7f764876c2c9e357c5bf16e9f5.jpg
FP: IMG_4243_jpeg_jpg.rf.d6cfb47ff9ad0fbe61359a153e7dfbc8.jpg
FP: manggarai_2_2023Y_11M_17D_23H_jpg.rf.7105c8f8d4292cb29cd0eb94f80a158d.jpg
FP: manggarai_2_2023Y_11M_18D_10H_jpg.rf.8ae45bf2231cb762020c10f22121f402.jpg
FP: manggarai_2_2023Y_11M_18D_9H_jpg.rf.a177245d79e9ab4b6ae81030029d3be0.jpg
FP: menteng_2023Y_11M_16D_16H_jpg.rf.1e4bd1dd8b4e6f829750b47874e33599.jpg
FP: menteng_2023Y_11M_17D_8H_jpg.rf.dea7da98187922c9861628b4ad435229.jpg
FP: new_SD3_jpg.rf.2b7c90e8df94aa69c4ea6963fd8a2908.jpg
FP: new_SD4_jpg.rf.7978d0d499b9ff61ffb3a79162533b42.jpg
FP: online110_jpg.rf.04560e6c86fd25964b4dae59b42ea032.jpg
FP: online144_jpg.rf.9c516824568dda462d719c4d75932a1f.jpg
FP: online167_jpg.rf.c49056f31f73555310741750090b38c5.jpg
FP: online254_jpg.rf.fcd55712cbba546163a339463f884724.jpg
FP: online30_jpg.rf.1095bca3cbb66baedc00c790